In [35]:
# import relevant libraries
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter
from statistics import mean

In [57]:
# api key is public, for the sake of this project
COIN_MARKET_CAP_API_TOKEN = '6d388297-0e7a-48ae-8955-64d099584672'

In [58]:
# code straight from CoinMarketCap documentation page. Works perfectly fine
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':'5000',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': COIN_MARKET_CAP_API_TOKEN,
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

In [146]:
# create a dataframe to store our information
my_columns = [
    'Ticker',
    'Name',
    'Price',
    'Number of Coins to Buy',
    'One-Hour Price Change',
    'One-Hour Price Change Percentile',
    'One-Day Price Change',
    'One-Day Price Change Percentile',
    'One-Week Price Change',
    'One-Week Price Change Percentile',
    'One-Month Price Change',
    'One-Month Price Change Percentile',
    'HQM Score'
]
crypto_dataframe = pd.DataFrame(columns=my_columns)
crypto_dataframe

,Ticker,Name,Price,Number of Coins to Buy,One-Hour Price Change,One-Hour Price Change Percentile,One-Day Price Change,One-Day Price Change Percentile,One-Week Price Change,One-Week Price Change Percentile,One-Month Price Change,One-Month Price Change Percentile,HQM Score


In [108]:
data['data'][0]['name']

'Bitcoin'

In [91]:
data['data'][0]['symbol']

'BTC'

In [148]:
# populate table with information
for index in range(0,len(data['data'])):
    crypto_dataframe = crypto_dataframe.append(
        pd.Series([
            data['data'][index]['symbol'],
            data['data'][index]['name'],
            data['data'][index]['quote']['USD']['price'],
            'N/A',
            data['data'][index]['quote']['USD']['percent_change_1h'],
            'N/A',
            data['data'][index]['quote']['USD']['percent_change_24h'],
            'N/A',
            data['data'][index]['quote']['USD']['percent_change_7d'],
            'N/A',
            data['data'][index]['quote']['USD']['percent_change_30d'],
            'N/A',
            'N/A'
        ],index = my_columns),
    ignore_index = True)
    
crypto_dataframe

,Ticker,Name,Price,Number of Coins to Buy,One-Hour Price Change,One-Hour Price Change Percentile,One-Day Price Change,One-Day Price Change Percentile,One-Week Price Change,One-Week Price Change Percentile,One-Month Price Change,One-Month Price Change Percentile,HQM Score
0,BTC,Bitcoin,47567.363069,N/A,1.208868,N/A,0.344680,N/A,21.436045,N/A,21.817190,N/A,N/A
1,ETH,Ethereum,1824.620631,N/A,0.773968,N/A,-0.923311,N/A,8.669198,N/A,50.307390,N/A,N/A
2,USDT,Tether,1.000562,N/A,-0.001269,N/A,-0.118465,N/A,-0.056710,N/A,0.159134,N/A,N/A
3,XRP,XRP,0.622173,N/A,0.856043,N/A,2.131743,N/A,41.719721,N/A,110.969541,N/A,N/A
4,ADA,Cardano,0.900271,N/A,1.080613,N/A,-2.740647,N/A,44.916785,N/A,191.653406,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4058,GLAD,Gladius Finance,4.749698,N/A,0.543730,N/A,-0.910365,N/A,-4.869852,N/A,0.000000,N/A,N/A
4059,VBETH,Venus BETH,35.490122,N/A,0.761533,N/A,-0.795514,N/A,0.000000,N/A,0.000000,N/A,N/A
4060,TTN,TheToken.Network,2073.942817,N/A,0.002999,N/A,-0.003367,N/A,13.139440,N/A,0.000000,N/A,N/A
4061,OT,Option Token,2.589555,N/A,1.234780,N/A,-0.926184,N/A,6.810599,N/A,0.000000,N/A,N/A


In [149]:
durations = [
    'One-Hour',
    'One-Day',
    'One-Week',
    'One-Month'
]

# ensure all appropriate columns have floats
for row in crypto_dataframe.index:
    for duration in durations:
        crypto_dataframe.loc[row, f'{duration} Price Change Percentile'] = 0
        
for row in crypto_dataframe.index:
    for duration in durations:
        change_col = f'{duration} Price Change'
        percentile_col = f'{duration} Price Change Percentile'
        crypto_dataframe.loc[row,percentile_col] = stats.percentileofscore(crypto_dataframe[change_col],crypto_dataframe.loc[row,change_col])

crypto_dataframe        

,Ticker,Name,Price,Number of Coins to Buy,One-Hour Price Change,One-Hour Price Change Percentile,One-Day Price Change,One-Day Price Change Percentile,One-Week Price Change,One-Week Price Change Percentile,One-Month Price Change,One-Month Price Change Percentile,HQM Score
0,BTC,Bitcoin,47567.363069,N/A,1.208868,66.8718,0.344680,51.8459,21.436045,57.0884,21.817190,42.124,N/A
1,ETH,Ethereum,1824.620631,N/A,0.773968,53.9257,-0.923311,30.5193,8.669198,41.3611,50.307390,57.6298,N/A
2,USDT,Tether,1.000562,N/A,-0.001269,30.4578,-0.118465,40.0812,-0.056710,24.8339,0.159134,29.3995,N/A
3,XRP,XRP,0.622173,N/A,0.856043,57.1253,2.131743,62.1216,41.719721,71.4251,110.969541,75.1415,N/A
4,ADA,Cardano,0.900271,N/A,1.080613,61.2355,-2.740647,22.8895,44.916785,73.2218,191.653406,85.8971,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4058,GLAD,Gladius Finance,4.749698,N/A,0.543730,45.6067,-0.910365,32.3776,-4.869852,18.3362,0.000000,26.4214,N/A
4059,VBETH,Venus BETH,35.490122,N/A,0.761533,52.572,-0.795514,34.9003,0.000000,25.8799,0.000000,26.4214,N/A
4060,TTN,TheToken.Network,2073.942817,N/A,0.002999,34.1989,-0.003367,42.2594,13.139440,46.7143,0.000000,26.4214,N/A
4061,OT,Option Token,2.589555,N/A,1.234780,76.6429,-0.926184,29.5348,6.810599,37.4354,0.000000,26.4214,N/A


In [150]:
# calculate HQM score using statistic module. mean of the 4 percentiles
for row in crypto_dataframe.index:
    momentum_percentiles = []
    for duration in durations:
        momentum_percentiles.append(crypto_dataframe.loc[row,f'{duration} Price Change Percentile'])
    crypto_dataframe.loc[row,'HQM Score'] = mean(momentum_percentiles)

crypto_dataframe

,Ticker,Name,Price,Number of Coins to Buy,One-Hour Price Change,One-Hour Price Change Percentile,One-Day Price Change,One-Day Price Change Percentile,One-Week Price Change,One-Week Price Change Percentile,One-Month Price Change,One-Month Price Change Percentile,HQM Score
0,BTC,Bitcoin,47567.363069,N/A,1.208868,66.8718,0.344680,51.8459,21.436045,57.0884,21.817190,42.124,54.4825
1,ETH,Ethereum,1824.620631,N/A,0.773968,53.9257,-0.923311,30.5193,8.669198,41.3611,50.307390,57.6298,45.859
2,USDT,Tether,1.000562,N/A,-0.001269,30.4578,-0.118465,40.0812,-0.056710,24.8339,0.159134,29.3995,31.1931
3,XRP,XRP,0.622173,N/A,0.856043,57.1253,2.131743,62.1216,41.719721,71.4251,110.969541,75.1415,66.4534
4,ADA,Cardano,0.900271,N/A,1.080613,61.2355,-2.740647,22.8895,44.916785,73.2218,191.653406,85.8971,60.811
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4058,GLAD,Gladius Finance,4.749698,N/A,0.543730,45.6067,-0.910365,32.3776,-4.869852,18.3362,0.000000,26.4214,30.6855
4059,VBETH,Venus BETH,35.490122,N/A,0.761533,52.572,-0.795514,34.9003,0.000000,25.8799,0.000000,26.4214,34.9434
4060,TTN,TheToken.Network,2073.942817,N/A,0.002999,34.1989,-0.003367,42.2594,13.139440,46.7143,0.000000,26.4214,37.3985
4061,OT,Option Token,2.589555,N/A,1.234780,76.6429,-0.926184,29.5348,6.810599,37.4354,0.000000,26.4214,42.5086


In [151]:
# sort the dataframe to get the currencies ranked by HQM scores
crypto_dataframe.sort_values('HQM Score',ascending=False,inplace=True)

crypto_dataframe.reset_index(inplace=True,drop=True)

In [152]:
crypto_dataframe = crypto_dataframe[:100]

In [153]:
# calculate number of shares given portfolio size
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

In [ ]:
portfolio_input()

In [ ]:
position_size = float(portfolio_size) / len(crypto_dataframe.index)
for i in range(0, len(crypto_dataframe['Ticker'])):
    crypto_dataframe.loc[i, 'Number of Coins to Buy'] = math.floor(position_size / crypto_dataframe['Price'][i])
crypto_dataframe

In [ ]:
# now simply store info in an excel file (xlsx)
writer = pd.ExcelWriter('crypto_momentum_strategy.xlsx', engine='xlsxwriter')
crypto_dataframe.to_excel(writer, sheet_name='Crypto Momentum Strategy', index = False)

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [ ]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Name', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Coins to Buy', integer_template],
                    'D': ['One-Hour Price Change', dollar_template],
                    'E': ['One-Hour Price Change Percentile', percent_template],
                    'F': ['One-Day Price Change', dollar_template],
                    'G': ['One-Day Price Change Percentile', percent_template],
                    'H': ['One-Week Price Change', dollar_template],
                    'I': ['One-Week Price Change Percentile', percent_template],
                    'J': ['One-Month Price Change', dollar_template],
                    'K': ['One-Month Price Change Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

# format the file as desired
for column in column_formats.keys():
    writer.sheets['Crypto Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Crypto Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)
    
# save file
writer.save()